## 错误与异常

错误可（至少）被分为两种：语法错误 和 异常。

[内部异常列表](https://docs.python.org/zh-cn/3/library/exceptions.html#bltin-exceptions)

### 语法错误

语法错误又称解析错误。

In [1]:
while True print('Hello world')

SyntaxError: invalid syntax (2884618176.py, line 1)

### 异常

即使语句或表达式使用了正确的语法，执行时仍可能触发错误。执行时检测到的错误称为 异常，异常不一定导致严重的后果。

In [4]:
10 * (1/0)

ZeroDivisionError: division by zero

In [3]:
4 + spam*3

NameError: name 'spam' is not defined

In [5]:
'2' + 2

TypeError: can only concatenate str (not "int") to str

### 异常的处理

try 语句的工作原理如下：

1. 首先，执行 try 子句 （try 和 except 关键字之间的（多行）语句）。

2. 如果没有触发异常，则跳过 except 子句，try 语句执行完毕。

3. 如果在执行 try 子句时发生了异常，则跳过该子句中剩下的部分。 如果异常的类型与 except 关键字后指定的异常相匹配，则会执行 except 子句，然后跳到 try/except 代码块之后继续执行。

4. 如果发生的异常与 except 子句 中指定的异常不匹配，则它会被传递到外层的 try 语句中；如果没有找到处理器，则它是一个 未处理异常 且执行将停止并输出一条错误消息。


一个 except 子句中的类匹配的异常将是该类本身的实例或其所派生的类的实例（但反过来则不可以 --- 列出派生类的 except 子句 不会匹配其基类的实例）。 例如，下面的代码将依次打印 B, C, D:

In [8]:
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

for cls in [B, C, D]:
    try:
        raise cls()
    except D:
        print("D", end=" ")
    except C:
        print("C", end=" ")
    except B:
        print("B", end=" ")
print()
for cls in [B, C, D]:
    try:
        raise cls()
    except B:
        print("B", end=" ")
    except D:
        print("D", end=" ")
    except C:
        print("C", end=" ")
print()

B C D 
B B B 


except 子句 可能会在异常名称后面指定一个变量。 这个变量将被绑定到异常实例，该实例通常会有一个存储参数的 args 属性。 为了方便起见，内置异常类型定义了 __str__() 来打印所有参数而不必显式地访问 .args。

In [9]:
try:
    raise Exception('spam', 'eggs')
except Exception as inst:
    print(type(inst))    # 异常的类型
    print(inst.args)     # 参数保存在 .args 中
    print(inst)          # __str__ 允许 args 被直接打印，
                         # 但可能在异常子类中被覆盖
    x, y = inst.args     # 解包 args
    print('x =', x)
    print('y =', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x = spam
y = eggs


BaseException 是所有异常的共同基类。它的一个子类， Exception ，是所有非致命异常的基类。不是 Exception 的子类的异常通常不被处理，因为它们被用来指示程序应该终止。它们包括由 sys.exit() 引发的 SystemExit ，以及当用户希望中断程序时引发的 KeyboardInterrupt 。

Exception 可以被用作通配符，捕获（几乎）一切。

处理 Exception 最常见的模式是打印或记录异常，然后重新提出（允许调用者也处理异常）:

In [10]:
import sys

try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error:", err)
except ValueError:
    print("Could not convert data to an integer.")
except Exception as err:
    print(f"Unexpected {err=}, {type(err)=}")
    raise

OS error: [Errno 2] No such file or directory: 'myfile.txt'


try ... except 语句具有可选的 else 子句，该子句如果存在，它必须放在所有 except 子句 之后。 它适用于 try 子句 **没有引发异常** (else需要没法发生异常时候才会执行到)但又必须要执行的代码。 例如:

In [11]:
for arg in sys.argv[1:]:
    try:
        f = open(arg, 'r')
    except OSError:
        print('cannot open', arg)
    else:
        print(arg, 'has', len(f.readlines()), 'lines')
        f.close()

cannot open -f
/data/tink/.local/share/jupyter/runtime/kernel-5aef9622-61b5-4689-9e6e-92582adf0eab.json has 13 lines


异常处理程序不仅会处理在 try 子句 中立刻发生的异常，还会处理在 try 子句 中调用（包括间接调用）的函数。 例如:

In [12]:
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
    print('Handling run-time error:', err)

Handling run-time error: division by zero


### 触发异常

raise 语句支持强制触发指定的异常。raise 唯一的参数就是要触发的异常。这个参数必须是异常实例或异常类（派生自 BaseException 类，例如 Exception 或其子类）。如果传递的是异常类，将通过调用没有参数的构造函数来隐式实例化。

In [ ]:
raise NameError('HiThere')

In [13]:
raise ValueError  # 'raise ValueError()' 的简化

ValueError: 

In [14]:
try:
    raise NameError('HiThere')
except NameError:
    print('An exception flew by!')
    raise #  raise 语句重新触发异常

An exception flew by!


NameError: HiThere

### 异常链

如果一个未处理的异常发生在 except 部分内，它将会有被处理的异常附加到它上面，并包括在错误信息中。

In [15]:
try:
    open("database.sqlite")
except OSError:

    raise RuntimeError("unable to handle error")

RuntimeError: unable to handle error

为了表明一个异常是另一个异常的直接后果， raise 语句允许一个可选的 from 子句:

```python
# exc 必须为异常实例或为 None。
raise RuntimeError from exc
```

In [16]:
def func():
    raise ConnectionError

try:
    func()
except ConnectionError as exc:
    raise RuntimeError('Failed to open database') from exc

RuntimeError: Failed to open database

使用 from None 表达禁用自动异常链:

In [17]:
try:
    open('database.sqlite')
except OSError:
    raise RuntimeError from None

RuntimeError: 

### 用户自定义异常

不论是以直接还是间接的方式，异常都应从 Exception 类派生。大多数异常命名都以 “Error” 结尾，类似标准异常的命名。

### final清理操作

final用于定义在所有情况下都必须要执行的清理操作。

In [20]:
try:
    raise KeyboardInterrupt
finally:
    print('Goodbye, world!')

Goodbye, world!


KeyboardInterrupt: 

如果存在 finally 子句，则 finally 子句是 try 语句结束前执行的最后一项任务。**不论 try 语句是否触发异常，都会执行 finally 子句**。以下内容介绍了几种比较复杂的触发异常情景：

- 如果执行 try 子句期间触发了某个异常，则某个 except 子句应处理该异常。如果该异常没有 except 子句处理，在 finally 子句执行后会被重新触发。

- except 或 else 子句执行期间也会触发异常。 同样，该异常会在 finally 子句执行之后被重新触发。

- 如果 finally 子句中包含 break、continue 或 return 等语句，异常将不会被重新引发。

- 如果执行 try 语句时遇到 break,、continue 或 return 语句，则 finally 子句在执行 break、continue 或 return 语句之前执行。

- 如果 finally 子句中包含 return 语句，则返回值来自 finally 子句的某个 return 语句的返回值，而不是来自 try 子句的 return 语句的返回值。


In [21]:
def bool_return():
    try:
        return True
    finally:
        return False
bool_return()


False

In [22]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        print("result is", result)
    finally:
        print("executing finally clause")

In [23]:
divide(2, 1)

result is 2.0
executing finally clause


In [24]:
divide(2, 0)

division by zero!
executing finally clause


In [25]:
divide("2", "1")

executing finally clause


TypeError: unsupported operand type(s) for /: 'str' and 'str'

### with用于预定义的清理操作

下例要打开一个文件，并输出文件内容：

```python
for line in open("myfile.txt"):
    print(line, end="")
```

上面代码缺点时候，没有是否打开的文件句柄资源。

[with](https://docs.python.org/zh-cn/3/reference/compound_stmts.html#with) 语句支持以及时、正确的清理的方式使用文件对象：

```python
with open("myfile.txt") as f:
    for line in f:
        print(line, end="")
```

语句执行完毕后，即使在处理行时遇到问题，都会关闭文件 f。和文件一样，支持预定义清理操作的对象会在文档中指出这一点。

### 追加异常信息

异常有一个 add_note(note) 方法接受一个字符串，并将其添加到异常的注释列表。标准的回溯在异常之后按照它们被添加的顺序呈现包括所有的注释。

In [27]:
try:
    raise TypeError('bad type')
except Exception as e:
    e.add_note('Add some information')
    e.add_note('Add some more information')
    raise

AttributeError: 'TypeError' object has no attribute 'add_note'

### 异常组

内置的 ExceptionGroup 打包了一个异常实例的列表，这样它们就可以一起被引发。它本身就是一个异常，所以它可以像其他异常一样被捕获。

In [28]:
def f():
    excs = [OSError('error 1'), SystemError('error 2')]
    raise ExceptionGroup('there were problems', excs)
f()

NameError: name 'ExceptionGroup' is not defined

In [ ]:
try:
    f()
except Exception as e:
    print(f'caught {type(e)}: e')

把异常收集到一个异常组时，我们可能想为各个错误添加上下文信息。在下文中，组中的每个异常都有一个说明，指出这个错误是什么时候发生的，示例如下：

In [29]:
def f():
    raise OSError('operation failed')

excs = []
for i in range(3):
    try:
        f()
    except Exception as e:
        e.add_note(f'Happened in Iteration {i+1}')
        excs.append(e)
raise ExceptionGroup('We have some problems', excs)

AttributeError: 'OSError' object has no attribute 'add_note'